# Node classification with Graph Convolutional Network (GCN)

In [33]:
import numpy as np

import mygene
import h5py
import pickle
import argparse
import networkx as nx
import seaborn as sns

import pandas as pd
import os

import stellargraph as sg
from stellargraph.layer import GCN
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
import sys

from stellargraph import StellarGraph
from stellargraph import datasets
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)

from stellargraph.layer import GCN, DeepGraphInfomax, GraphSAGE, GAT, APPNP, HinSAGE
from stellargraph.utils import plot_history

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.manifold import TSNE
from sklearn.metrics import average_precision_score
from IPython.display import display, HTML

import tensorflow as tf
from scipy.sparse import csr_matrix, lil_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import Model
from keras.models import Sequential
from keras.layers import Dense

from imblearn.over_sampling import SMOTE

In [35]:
network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/HPRD_network.tsv', sep='\t')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/HPRD_features_complete.tsv', sep='\t', index_col='gene')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/HPRD_labels_semisupervised.tsv', sep='\t', index_col='gene')

In [36]:
# Mutations 0:48
# CNA 16:64
# DNA Methylation 0:32 e 16:48
# Gene Expression 0:16 e 16:48


#features.drop(features.iloc[:, 0:16], inplace = True, axis = 1)
#features.drop(features.iloc[:, 16:48], inplace = True, axis = 1)
#features

In [37]:
# Transformar as labels boleanas em 0/1
labels["label"].replace({False: 0, True: 1}, inplace=True)

# Transformar as labels vazias em -1
labels["label"] = labels.label.fillna(-1)

In [38]:
G = StellarGraph(edges=network, nodes=features)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 9438, Edges: 36844

 Node types:
  default: [9438]
    Features: float32 vector, length 68
    Edge types: default-default->default

 Edge types:
    default-default->default: [36844]
        Weights: all 1 (default)
        Features: none


In [39]:
series_classes = labels['label']

series_classes.value_counts(dropna = False).to_frame()

,label
0.0,4873
-1.0,3772
1.0,793


In [40]:
# Divisão treino/teste

labeled_data = labels[labels['label'] != -1]
labeled_data = labeled_data.sample(frac=1)

# Um conjunto de teste é utilizado em todos as redes, para isso foi selecionado posteriormente uma lista de genes que é comum à todas as redes
test_set = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final_v2/test_set_final.tsv', sep='\t')

# Método para separar os dados de treino a partir do test_set
# É criada uma nova coluna comparando os genes do banco de teste pré-selecionado com os dados rotulados totais da rede

labeled_data['treino'] = labeled_data.index.isin(test_set['gene'])
labeled_data.treino.value_counts().to_frame()

# Sempre deve existir 1134 True, pois são genes que existem em todas as redes, logo, o restante é separado para treinamento

labeled_train_temp = labeled_data[labeled_data['treino'] == False]
labeled_test_temp = labeled_data[labeled_data['treino'] == True]

labeled_train = labeled_train_temp.drop(columns=['treino'])
labeled_test = labeled_test_temp.drop(columns=['treino'])

print("Train: ", len(labeled_train))
print("Test: ", len(labeled_test))
print("\nTotal: ", len(labeled_train)+len(labeled_test))

Train:  4532
Test:  1134

Total:  5666


In [41]:
# Difinição da função de custo Focal Loss

import dill

from keras import backend as K

def binary_focal_loss(gamma=2., alpha=.25):

    def binary_focal_loss_fixed(y_true, y_pred):
     
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss

    return binary_focal_loss_fixed

In [42]:
target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(labeled_train)
test_targets = target_encoding.transform(labeled_test)



generator = FullBatchNodeGenerator(G, method="gcn")

Using GCN (local pooling) filters...


In [43]:
from sklearn.model_selection import KFold

num_folds = 5

auc_pr_per_fold = []
loss_per_fold = []

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
for train, test in kfold.split(labeled_train.index, train_targets):

  train_gen = generator.flow(labeled_train.index[train], train_targets[train])

  gcn = GCN(
    layer_sizes=[64, 64], activations=["relu", "relu"], generator=generator, dropout=0.1
  )

  x_inp, x_out = gcn.in_out_tensors()

  predictions = layers.Dense(units=train_targets.shape[1], activation="sigmoid")(x_out)

  model = Model(inputs=x_inp, outputs=predictions)
  model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    #loss=losses.binary_crossentropy,
    loss=[binary_focal_loss(gamma=0.5, alpha=0.5)],
    metrics=["acc", metrics.AUC(curve="ROC", name="auc_roc"), metrics.AUC(curve="PR", name="auc_pr")],
  )

  val_gen = generator.flow(labeled_train.index[test], train_targets[test])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = model.fit(
    train_gen,
    epochs=500,
    validation_data=val_gen,
    verbose=2,
    shuffle=False,
  )

  # Generate generalization metrics
  #val_gen = generator.flow(labeled_train.index[test], train_targets[test])
  scores = model.evaluate(val_gen, verbose=0)
  print('\n')
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[3]} of {scores[3]*100}%')
  auc_pr_per_fold.append(scores[3] * 100)
  loss_per_fold.append(scores[0])

  # Save history to csv
  hist_df = pd.DataFrame(history.history)

  hist_csv_file = f'fold{fold_no}_hprd_6a.csv'
  with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
  f.close()

  # Increase fold number
  fold_no = fold_no + 1

# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(auc_pr_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - AUC PR: {auc_pr_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> AUC PR: {np.mean(auc_pr_per_fold)} (+- {np.std(auc_pr_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/500
1/1 - 4s - loss: 934.5020 - acc: 0.1399 - auc_roc: 0.6244 - auc_pr: 0.2022 - val_loss: 199.3744 - val_acc: 0.8600 - val_auc_roc: 0.2693 - val_auc_pr: 0.0894
Epoch 2/500
1/1 - 0s - loss: 796.5490 - acc: 0.8601 - auc_roc: 0.3099 - auc_pr: 0.0949 - val_loss: 177.5501 - val_acc: 0.8600 - val_auc_roc: 0.2870 - val_auc_pr: 0.0917
Epoch 3/500
1/1 - 0s - loss: 707.2839 - acc: 0.8601 - auc_roc: 0.3226 - auc_pr: 0.0969 - val_loss: 162.9277 - val_acc: 0.8600 - val_auc_roc: 0.2821 - val_auc_pr: 0.0911
Epoch 4/500
1/1 - 0s - loss: 643.7502 - acc: 0.8601 - auc_roc: 0.3251 - auc_pr: 0.0976 - val_loss: 161.7511 - val_acc: 0.8600 - val_auc_roc: 0.2808 - val_auc_pr: 0.0912
Epoch 5/500
1/1 - 0s - loss: 631.5494 - acc: 0.8601 - auc_roc: 0.3207 - auc_pr: 0.0971 - val_loss: 169.5254 - val_acc: 0.8600 - val_auc_roc: 0.2813 - val_auc_pr: 0.0913
Epoch 6/500
1/1 - 0s - loss: 656.0757 - acc: 0.8601 - auc_

In [44]:
# Execução para os dados de teste

test_gen = generator.flow(labeled_test.index, test_targets)
test_metrics = model.evaluate(test_gen)

print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

1/1 [==============================] - 0s 107ms/step - loss: 131.5420 - acc: 0.8836 - auc_roc: 0.8397 - auc_pr: 0.5884

Test Set Metrics:
	loss: 131.5420
	acc: 0.8836
	auc_roc: 0.8397
	auc_pr: 0.5884


In [45]:
all_nodes = series_classes.index
all_gen = generator.flow(all_nodes)
all_predictions = model.predict(all_gen)

In [46]:
node_predictions = target_encoding.inverse_transform(all_predictions.squeeze())
df = pd.DataFrame({"Predicted": node_predictions, "True": series_classes})

#df.head(20)

print(df['True'].value_counts(), "\n")
df['Predicted'].value_counts()

 0.0    4873
-1.0    3772
 1.0     793
Name: True, dtype: int64 



0.0    8324
1.0    1114
Name: Predicted, dtype: int64

In [47]:
# Salvar a variável all_predictions e a node_predictions para comparar
# Usar a all_predictions para gerar gráficos da fig3 do artigo, sobre distribuição de nós 

df_predictions = pd.DataFrame({"percent": all_predictions.squeeze(), "binary": node_predictions, "true": series_classes})

# Save to csv
pred_csv_file = f'predictions_hprd_6a.csv'
with open(pred_csv_file, mode='w') as f:
    df_predictions.to_csv(f)
f.close()